Syntax natürlicher Sprachen, WS 2023/24

# 03 - Übung (Lösung)

In [1]:
import nltk
from nltk.tree import Tree

## Aufgabe 1 -  Syntaktische Ambiguität

#### Gegeben sei folgende CFG-Grammatik für einen bekannten Beispielsatz aus einem Film der Marx Brothers (vgl. https://www.nltk.org/book/ch08.html#ex-marx-elephant). Führen Sie das Skript aus und beantworten Sie im Anschluss die untenstehenden Fragen.

In [2]:
sent = "I shot an elephant in my pajamas"

grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> DET N | DET N PP | PRON
    VP -> V NP | VP PP
    PP -> P NP
    PRON -> 'I'
    DET -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
print("\nLänge der vom Parser wiedergegebenen Sequenz: ", len(trees), "\n")

for tree in trees:
    tree.pretty_print(unicodelines=True)


Länge der vom Parser wiedergegebenen Sequenz:  2 

      S                                       
 ┌────┴──────────────┐                         
 │                   VP                       
 │         ┌─────────┴──────────┐              
 │         VP                   PP            
 │    ┌────┴───┐            ┌───┴───┐          
 NP   │        NP           │       NP        
 │    │    ┌───┴─────┐      │   ┌───┴─────┐    
PRON  V   DET        N      P  DET        N   
 │    │    │         │      │   │         │    
 I   shot  an     elephant  in  my     pajamas

      S                                   
 ┌────┴──────────┐                         
 │               VP                       
 │    ┌──────────┴──────┐                  
 │    │                 NP                
 │    │    ┌─────┬──────┴───┐              
 │    │    │     │          PP            
 │    │    │     │      ┌───┴───┐          
 NP   │    │     │      │       NP        
 │    │    │     │      │   ┌───┴─

#### 1. Warum gibt die parse-Funktion des Skripts eine Sequenz zurück?


In [17]:
# Zu einer Eingabe können im allgemeinen mehrere Parse-Bäume gefunden werden (verschiedene Analysen gemäß der Grammatik).
# Deshalb enthält das Rückgabeobjekt der parse()-Methode eine Liste, die die gefundenen Tree-Objekte enthält.

#### 2. Worin liegen die Unterschiede zwischen den ausgegebenen Bäumen?


In [ ]:
# Ambiguität! 
# Die Präpositionalphrase 'in my pajamas' ist einmal Teil der Verbalphrase und einmal Teil der Nominalphrase.

#### 3. Warum ist der gegebene Satz *I shot an elephant in my pajamas* syntaktisch ambig?

In [18]:
# PP-Attachment-Ambiguität:
# Aus der Oberflächensyntax ist nicht ersichtlich, welche syntakische Funktion die PP einnimmt. 

# Im ersten Fall modifiziert die PP die im Verb ausgedrückete Tätigkeit (Adverbial).
# Im zweiten Fall stellt die PP eine Beifügung zum Substantiv dar (Attribut).


#### 4. Welche Regeln ermöglichen die beiden Analysen? Wie unterscheiden sich die beiden Regeln?

In [ ]:
# VP -> VP PP 
# = rekursive Regel für PP-Adjunktion von Adverbialen an die VP

# NP -> DET N PP 
# = flache (nicht-rekursive) Regel für die Adjunktion von genau einer PP als Attribut an die NP

## Aufgabe 2 - Transformation in X-Bar-Struktur

#### Ergänzen Sie die gegebene CFG mit X-Bar-Struktur-Regeln, um folgenden Satz linguistisch korrekt zu parsen:

In [3]:
sent = "I had shot a very very big elephant in my pajamas"

In [4]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP

    NP -> DET NOM | PRON
    NOM -> ADJP NOM | NOM PP
    NOM -> N

    ADJP -> PRT ADJP
    ADJP -> ADJ

    VP -> AUX VERBAL
    VERBAL -> VERBAL PP
    VERBAL -> V NP

    PP -> P NP

    PRON -> 'I'
    DET -> 'a' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
    PRT -> 'very'
    ADJ -> 'big'
    AUX -> 'had'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

          S                                                           
 ┌────────┴───────────────┐                                            
 │                        VP                                          
 │    ┌───────────────────┴──────────┐                                 
 │    │                            VERBAL                             
 │    │             ┌────────────────┴──────────────────┐              
 │    │           VERBAL                                │             
 │    │   ┌─────────┴─────┐                             │              
 │    │   │               NP                            │             
 │    │   │    ┌──────────┴────┐                        │              
 │    │   │    │              NOM                       │             
 │    │   │    │          ┌────┴─────────────┐          │              
 │    │   │    │         ADJP                │          PP            
 │    │   │    │    ┌─────┴────┐             │      ┌───┴───┐          

In [ ]:
# Erläuterung:
# hier treten zwei Arten syntaktischer Ambiguität auf:
# 1. PP-Attachment
# 2. Skopus der Adjektivphrase


## Aufgabe 3 - Phrasenstrukturgrammatik mit X-Bar-Regeln

### a) Schreiben Sie zunächst eine einfache kontextfreie Grammatik, die die folgenden beiden Sätze erkennt:

In [6]:
sents = [
    "der Mann gibt der Frau das Buch",
    "die Frau schenkt der Frau das Buch"
]

In [7]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> DET N
    VP -> V NP NP

    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    V -> "gibt" | "schenkt"
""")

parser = nltk.ChartParser(grammar)
for sent in sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

                   S                       
     ┌─────────────┴───────┐                
     │                     VP              
     │        ┌────────┬───┴────────┐       
     NP       │        NP           NP     
 ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
DET      N    V   DET      N   DET      N  
 │       │    │    │       │    │       │   
der     Mann gibt der     Frau das     Buch

                      S                       
     ┌────────────────┴───────┐                
     │                        VP              
     │          ┌─────────┬───┴────────┐       
     NP         │         NP           NP     
 ┌───┴───┐      │     ┌───┴───┐    ┌───┴───┐   
DET      N      V    DET      N   DET      N  
 │       │      │     │       │    │       │   
die     Frau schenkt der     Frau das     Buch



### b) Erweiterung für ein- und zweistellige Verben

#### Bis jetzt akzeptiert die Grammatik lediglich dreistellige Verben. Erweitern Sie sie so, dass auch Verben mit weniger als zwei Objekten korrekte Verbalphrasen bilden können.

#### Folgende Sätze sollten akzeptiert werden:

In [8]:
sents = [
    "der Mann schläft",
    "das Buch gefällt der Frau",
    "die Frau kennt das Buch"
]

In [9]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> DET N
# neu:
    VP -> V NP NP | V NP | V

    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    V -> "gibt" | "schenkt" | "schläft" | "gefällt" | "kennt"
""")

parser = nltk.ChartParser(grammar)
for sent in sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

         S          
     ┌───┴──────┐    
     NP         VP  
 ┌───┴───┐      │    
DET      N      V   
 │       │      │    
der     Mann schläft

                S                
     ┌──────────┴─────┐           
     │                VP         
     │          ┌─────┴───┐       
     NP         │         NP     
 ┌───┴───┐      │     ┌───┴───┐   
DET      N      V    DET      N  
 │       │      │     │       │   
das     Buch gefällt der     Frau

               S               
     ┌─────────┴────┐           
     │              VP         
     │         ┌────┴───┐       
     NP        │        NP     
 ┌───┴───┐     │    ┌───┴───┐   
DET      N     V   DET      N  
 │       │     │    │       │   
die     Frau kennt das     Buch



### c) Erweiterung für rekursive Adjektiv-Adjunkte

#### Erweitern Sie die Grammatik nun derart, dass Nominalphrasen auch Adjektive enthalten dürfen – und zwar beliebig viele. Beachten Sie dabei, dass andererseits nur genau ein Determinierer in der NP-Phrase enthalten sein darf (und zwar als erstes Element).

#### Verwenden Sie `NOM` als Label für die nominale Zwischenebene (X' in X-Bar-Schema).

#### Folgende Sätze sollten akzeptiert werden:

In [10]:
sents = [
    "die kluge schöne Frau kennt das Buch",
    "der schöne kluge Mann gibt der Frau das dicke Buch",
    "das dicke schöne kluge Buch schläft"
]

In [11]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
# neu:
    NP -> DET NOM 
    NOM -> ADJ NOM | N

    VP -> V NP NP | V NP | V

    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    V -> "gibt" | "schenkt" | "schläft" | "gefällt" | "kennt"
    ADJ -> "kluge" | "schöne" | "dicke"
""")

parser = nltk.ChartParser(grammar)
for sent in sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

                          S                     
           ┌──────────────┴──────────┐           
           NP                        │          
 ┌─────────┴────┐                    │           
 │             NOM                   VP         
 │    ┌─────────┴─────┐         ┌────┴───┐       
 │    │              NOM        │        NP     
 │    │         ┌─────┴───┐     │    ┌───┴───┐   
 │    │         │        NOM    │    │      NOM 
 │    │         │         │     │    │       │   
DET  ADJ       ADJ        N     V   DET      N  
 │    │         │         │     │    │       │   
die kluge     schöne     Frau kennt das     Buch

                               S                                  
            ┌──────────────────┴────────────┐                      
            NP                              VP                    
 ┌──────────┴────┐             ┌────────┬───┴─────────┐            
 │              NOM            │        │             NP          
 │    ┌──────────┴─

#### Erweiterte Negativtests: Testen Sie auch, ob Ihre Grammatik für die folgenden (ungrammatischen) Sätze keine Ableitung liefert:

In [12]:
neg_sents = [
    "der der dicke der Mann gibt der Frau das Buch"
]

parser = nltk.ChartParser(grammar)
for sent in neg_sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

no parse found for: der der dicke der Mann gibt der Frau das Buch


In [13]:
# Erweiterte Lösung: 
#Folgende Grammatik mit rekursiven NP-Regeln ohne die nominale Zwischenebene (`NOM`) erkennt dagegen diese ungrammatischen Sätze 
#und ist insofern keine korrektes Modell der NP-Phrasenstruktur des Deutschen:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> N | ADJ NP | DET NP 
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> V NP NP
    V -> "gibt" | "schenkt"
    ADJ -> "dicke"
""")

parser = nltk.ChartParser(grammar)
for sent in neg_sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

                           S                                 
          ┌────────────────┴─────────────────┐                
          NP                                 │               
 ┌────────┴────┐                             │                
 │             NP                            │               
 │   ┌─────────┴───┐                         │                
 │   │             NP                        VP              
 │   │    ┌────────┴───┐        ┌────────┬───┴────────┐       
 │   │    │            NP       │        NP           NP     
 │   │    │        ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
 │   │    │        │       NP   │    │       NP   │       NP 
 │   │    │        │       │    │    │       │    │       │   
DET DET  ADJ      DET      N    V   DET      N   DET      N  
 │   │    │        │       │    │    │       │    │       │   
der der dicke     der     Mann gibt der     Frau das     Buch



### d) Erweiterung um Präpositionalphrasen

#### Erweitern Sie die Grammatik nun derart, dass sowohl Nominalphrasen als auch Verbalphrasen durch Präpositionalphrasen modifiziert werden können.

#### Folgende Sätze sollten akzeptiert werden. Stellen Sie zudem sicher, dass für den 2. Satz zwei Analysen generiert werden (PP-Attachment-Ambiguität).

In [14]:
sents = [
    "der Mann schläft neben dem Buch",
    "die Frau schenkt dem Mann das Buch auf der Tanzfläche"
]

In [15]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> DET NOM 
    NOM -> ADJ NOM | N
#neu:
    NOM -> NOM PP

    VP -> V NP NP | V NP | V
#neu (hier ohne X-Bar-Zwischenebene VERBAL):
    VP -> VP PP

#neu:
    PP -> P NP

    DET -> "der" | "die" | "das" | "dem" | "der"
    N -> "Mann" | "Frau" | "Buch" | "Tanzfläche"
    V -> "gibt" | "schenkt" | "schläft" | "gefällt" | "kennt"
    ADJ -> "kluge" | "schöne" | "dicke"
    P -> "neben" | "auf"
""")

parser = nltk.ChartParser(grammar)
for sent in sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

                S                          
     ┌──────────┴───────────┐               
     │                      VP             
     │          ┌───────────┴───┐           
     │          │               PP         
     │          │      ┌────────┴───┐       
     NP         │      │            NP     
 ┌───┴───┐      │      │        ┌───┴───┐   
 │      NOM     VP     │        │      NOM 
 │       │      │      │        │       │   
DET      N      V      P       DET      N  
 │       │      │      │        │       │   
der     Mann schläft neben     dem     Buch

                              S                                      
     ┌────────────────────────┴────┐                                  
     │                             VP                                
     │                        ┌────┴────────────────┐                 
     │                        VP                    PP               
     │          ┌─────────┬───┴────────┐        ┌───┴───┐            

#### Testen Sie abschließend, ob Ihre Grammatik weiterhin für alle Sätze eine korrekte Analyse liefert:

In [16]:
all_sents = [
    "der Mann gibt der Frau das Buch",
    "die Frau schenkt der Frau das Buch",
    "der Mann schläft",
    "das Buch gefällt der Frau",
    "die Frau kennt das Buch",
    "die kluge schöne Frau kennt das Buch",
    "der schöne kluge Mann gibt der Frau das dicke Buch",
    "das dicke schöne kluge Buch schläft",
    "der Mann schläft neben dem Buch",
    "die Frau schenkt dem Mann das Buch auf der Tanzfläche"
]

In [17]:
parser = nltk.ChartParser(grammar)
for sent in all_sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

                   S                       
     ┌─────────────┴───────┐                
     │                     VP              
     │        ┌────────┬───┴────────┐       
     NP       │        NP           NP     
 ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
 │      NOM   │    │      NOM   │      NOM 
 │       │    │    │       │    │       │   
DET      N    V   DET      N   DET      N  
 │       │    │    │       │    │       │   
der     Mann gibt der     Frau das     Buch

                      S                       
     ┌────────────────┴───────┐                
     │                        VP              
     │          ┌─────────┬───┴────────┐       
     NP         │         NP           NP     
 ┌───┴───┐      │     ┌───┴───┐    ┌───┴───┐   
 │      NOM     │     │      NOM   │      NOM 
 │       │      │     │       │    │       │   
DET      N      V    DET      N   DET      N  
 │       │      │     │       │    │       │   
die     Frau schenkt der     Frau d